# **<center><font style="color:rgb(100,109,254)">Module 7: Physics + Computer Vision</font> </center>**

<center>
    <img src='https://drive.google.com/uc?export=download&id=1fZcQOIeX7OVJhsmMH5COSOzbBAxA28UG' width=800> 
</center>
    

## **<font style="color:rgb(134,19,348)"> Module Outline </font>**

The module can be split into the following parts:

- *Lesson 1: Introduction to Pymunk*

- ***Lesson 2:* Integrating Pymunk with OpenCV** *(This Tutorial)*

- *Lesson 3: Build the Final Application*


**Please Note**, these Jupyter Notebooks are not for sharing; do read the Copyright message below the Code License Agreement section which is in the last cell of this notebook.
-Taha Anwar

Alright, let's get started.

### **<font style="color:rgb(134,19,348)"> Import the Libraries</font>**

First, we will import the required libraries.

In [1]:
import cv2
import pymunk
import numpy as np
from time import time

### **<font style="color:rgb(134,19,348)">Create a Function to Convert Coordinates</font>**


You may have noticed (in the previous lesson) that in Pymunk, the y-axis value decreases in a downwards direction whereas it increases in OpenCV, so we will create a function that will take care of this conflict and will convert points from one coordinate system to another.

<center><img src='https://drive.google.com/uc?export=download&id=1wGxxg1lpZJSeRr2k-oqEHcPi023gphAn' width=600></center>

In [2]:
def convertPoints(frame, coordinates):
    '''
    This function will convert points from the coordinate system (used in OpenCV) to the
    coordinate system (used in Pymunk) and viceversa.
    Args:
        coordinates: The x and y coordinates that are about to be converted.
    Returns:
        converted_coordinates: The x and y coordinates after the conversion.
    '''
    
    # Get the height and width of the frame.
    height, width, _ = frame.shape
    
    # Get the x and y coordinates.
    x, y = coordinates
    
    # Convert the coordinates.
    converted_coordinates = int(x), int(height-y)
    
    # Return the converted coordinates.
    return converted_coordinates

## **<font style="color:rgb(134,19,348)">Create a Function to Make Balls</font>**

Now we will create a function **`createBall()`**, that will make balls with actual physical properties (mass, inertia, friction, and everything) utilizing Pymunk's [**`moment_for_circle()`**](http://www.pymunk.org/en/latest/pymunk.html#pymunk.moment_for_circle) function (to calculate the ball's moment of inertia), the [**`Body`**](http://www.pymunk.org/en/latest/pymunk.html#pymunk.Body) class (to create a DYNAMIC body for the ball) and the [**`Circle`**](http://www.pymunk.org/en/latest/pymunk.html#pymunk.Circle) class (to create a circular shape for the ball).

**Note that;** Pymunk allows us to create three different types of bodies, which are:

1. [DYNAMIC](http://www.pymunk.org/en/latest/pymunk.html#pymunk.Body.DYNAMIC) (Default type); These bodies react to collisions, are affected by forces and gravity, and have a finite amount of mass. These are the type of bodies that you want the physics engine to simulate for you.


2. [KINEMATIC](http://www.pymunk.org/en/latest/pymunk.html#pymunk.Body.KINEMATIC); These are bodies that are controlled from your code instead of inside the physics engine. They aren't affected by gravity and they have an infinite amount of mass so they don't react to collisions or forces with other bodies. Kinematic bodies are controlled by setting their velocity, which will cause them to move. Good examples of kinematic bodies might include things like moving platforms. 


3. [STATIC](http://www.pymunk.org/en/latest/pymunk.html#pymunk.Body.STATIC); These are the bodies that never (or rarely) move. Using static bodies for things like terrain offers a big performance boost over other body types- because Pymunk doesn't need to check for collisions between static objects and it never needs to update their collision information. Additionally, because static bodies don't move, Pymunk knows it's safe to let objects that are touching or joined to them fall asleep.

In [4]:
def createBall(frame, ball_position, radius):
    '''
    This function creates a ball with actual physical properties.
    Args:
        frame:         A frame/image from a real-time webcam feed. 
        ball_position: The postion of the ball to be created.
        radius:        The radius of the ball to be created.
    Returns:
        body:       The body of the ball created.
        shape:      The shape of the ball created.
        ball_color: The color of the ball created.
    '''
    
    # Calculate the moment of inertia for the ball.
    # Moment of inertia is the property of the body due to which it resists angular acceleration.
    # Where angular acceleration is the rate with which its angular velocity changes with time.
    moment = pymunk.moment_for_circle(mass=1, inner_radius=0, outer_radius=radius)
    
    # Create a dynamic body for the ball.
    body = pymunk.Body(mass=1, moment=moment, body_type=pymunk.Body.DYNAMIC)
    
    # Convert the points from OpenCV coordinates system to the Pymunk coordinates system.
    body.position = convertPoints(frame, ball_position)
    
    # Create a circle shape and attach it to the body. 
    shape = pymunk.Circle(body, radius)
    
    # Set the density of the shape.
    # The density of a body is defined as the amount of mass contained per unit volume of the body.
    shape.density = 0.0001
    
    # Set the elasticity of the shape.
    # Elasticity is the ability of a deformed body to return to its original shape and 
    # size, when the forces causing the deformation are removed.
    shape.elasticity = 0.98
    
    # Set the friction of the shape.
    # Friction is the force that always opposes the motion of one body over the otherbody in contact with it.
    shape.friction = 1.0
    
    # Get a random color for the ball.
    ball_color = tuple(np.random.choice(range(256), size=3))
    
    # Return the ball's body, shape, and color.
    return (body, shape), ball_color

## **<font style="color:rgb(134,19,348)">Create a Function to Draw Balls</font>**

Now we will create a function **`drawBalls()`**, that will take in, all the created balls' positions, sizes, and colors, and will simply draw circles on a real-time webcam feed using [**`cv2.circle()`**](https://docs.opencv.org/4.x/d6/d6e/group__imgproc__draw.html#gaf10604b069374903dbd0f0488cb43670) function.

In [5]:
def drawBalls(frame, balls_to_draw):
    '''
    This function draws balls on a frame/image.
    Args:
        frame:         The frame/image to draw balls on, from a real-time webcam feed. 
        balls_to_draw: A list containing the bodies, radiuses, and the colors of the balls to draw.
    Returns:
        frame: The frame/image with the balls drawn, from a real-time webcam feed.
    '''
    
    # Iterate over the balls to draw.
    for ball_to_draw in balls_to_draw:
        
        # Get the ball's body, shape, and color.
        ball_body, _, radius, ball_color = ball_to_draw
        
        # Get the RGB values of the ball color.
        r, g, b = ball_color
        
        # Convert the ball's position from the Pymunk's coordinates system to the OpenCV coordinates system.
        x, y = convertPoints(frame, ball_body.position)
        
        # Draw the ball on the frame.
        cv2.circle(img=frame, center=(x,y), radius=radius,  color=(int(b), int(g), int(r)), thickness=-1)
    
    # Return the frame with the balls drawn.
    return frame

## **<font style="color:rgb(134,19,348)">Create a Function to Make Obstacles</font>**

We will also need some STATIC obstacles in our simulation to properly check whether the balls we have created, have the required physical properties. So now we will create a function **`createObstacle()`** that will add some static line segments (obstacles for the balls) in the simulation utilizing Pymunk's [**`Body`**](http://www.pymunk.org/en/latest/pymunk.html#pymunk.Body) class and the [**`Segment`**](http://www.pymunk.org/en/latest/pymunk.html#pymunk.Segment) class.

In [6]:
def createObstacle(space, starting_point, ending_point, thickness):
    '''
    This function creates a static line segment that will act as an obstacle for the objects.
    Args:
        space:          The space which contains the simulation.
        starting_point: The starting point of the line segment, that is to be created.
        ending_point:   The ending point of the line segment, that is to be created.
        thickness:      The thickness of the line segment, that is to be created.
    Returns:
        starting_point: The starting point of the line segment, that is created.
        ending_point:   The ending point of the line segment, that is created.
        thickness:      The thickness of the line segment, that is created.
    '''
    
    # Create a static type body 
    segment_body = pymunk.Body(body_type=pymunk.Body.STATIC)
    
    # Create a line segment shape and attach it to the body. 
    segment_shape = pymunk.Segment(body=segment_body, a=starting_point, b=ending_point, radius=thickness/2)
    
    # Set the density of the shape. 
    segment_shape.density = 1
    
    # Set the elasticity of the shape.
    segment_shape.elasticity = 0.98
    
    # Set the friction of the shape.
    segment_shape.friction = 1.0
    
    # Add both the body and the shape of the line segment to the simulation.
    space.add(segment_body, segment_shape)
    
    # Return the starting point, ending point, and the thickness of the line segment.
    return (starting_point, ending_point), thickness

## **<font style="color:rgb(134,19,348)">Create a Function to Draw Obstacles</font>**

And now we will create a function **`drawObstacles()`**, that will utilize the [**`cv2.line()`**](https://docs.opencv.org/4.x/d6/d6e/group__imgproc__draw.html#ga7078a9fae8c7e7d13d24dac2520ae4a2) function to draw the line segments (obstacles) on a real-time webcam feed.

In [7]:
def drawObstacles(frame, segments_to_draw):
    '''
    This function draws the line segments on a frame/image.
    Args:
        frame:            The frame/image to draw segments on, from a real-time webcam feed. 
        segments_to_draw: A list containing the starting points, ending points, and the thicknesses of the segments to draw.
    Returns:
        frame: The frame/image with the segments drawn, from a real-time webcam feed.
    '''
    
    # Iterate over the segments to draw.
    for segment_to_draw in segments_to_draw:
        
        # Get the starting point, ending point, and the thickness of the segment, we are iterating upon.
        (starting_point, ending_point), thickness = segment_to_draw
        
        # Convert the starting point and the ending point from the 
        # Pymunk's coordinates system to the OpenCV coordinates system.
        starting_point = convertPoints(frame, starting_point)
        ending_point = convertPoints(frame, ending_point)
        
        # Draw the line segment on the frame.
        cv2.line(img=frame, pt1=starting_point, pt2=ending_point, color=(0,0,255),
                 thickness=thickness)
    
    # Return the frame with the line segments drawn.
    return frame

Now we will utilize the functions **`createBall()`**, and **`drawBalls()`** created above to add a new ball into the simulation every time user presses the **`b`** key and will utilize the functions **`createObstacle()`**, and **`drawObstacles()`** to create a few obstacles for the balls on a real-time webcam feed.

In [10]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0)
camera_video.set(3,1280)
camera_video.set(4,960)

# Get the height and width of the webcam video.
height, width = (int(camera_video.get(cv2.CAP_PROP_FRAME_HEIGHT)), 
                 int(camera_video.get(cv2.CAP_PROP_FRAME_WIDTH)))

# Create a named window for resizing purposes.
cv2.namedWindow('Webcam Feed', cv2.WINDOW_NORMAL)

# Create a Space which will contain the simulation.
space = pymunk.Space()

# Set the gravity.
space.gravity = 500, -500 

# Initialize a dictionary that will contain the balls and segments to draw.
to_draw = {'Balls': [], 'Segments': []}

# Create line segments and append them into the list inside the dictionary.
to_draw['Segments'].append(createObstacle(space, starting_point=(width//1.1, height//1.2),
                                         ending_point=(width//1.8, height//3), thickness=20))
to_draw['Segments'].append(createObstacle(space, starting_point=(width//10, height//2),
                             ending_point=(width//2, height//8), thickness=20))

# Initialize a variable to store the frame count.
frame_count = 0

# Get the start time.
start_time = time()

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
   
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then 
    # continue to the next iteration to read the next frame.
    if not ok:
        continue
        
    # Increment the frame counter.
    frame_count += 1
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)
    
    # Calaculate average frames per second.
    ##################################################################################################
    
    # Get the current time.
    curr_time = time()
    
    # Check if the difference between the start and current time > 0 to avoid division by zero.
    if (curr_time - start_time) > 0:
    
        # Calculate the number of frames per second.
        frames_per_second = frame_count // (curr_time - start_time)
        
        # Write the calculated number of frames per second on the frame. 
        cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10, int(width/25)),
                    cv2.FONT_HERSHEY_PLAIN, int(width/300), (0, 255, 0), int(width/200))
    
    ##################################################################################################
    
    # Iterate over the created balls.
    for ball in to_draw['Balls']:
        
        # Get the ball's body and shape.
        ball_body, ball_shape, _, _ = ball
        
        # Check if the ball's current position y-coordinate is < 0, which means that ball has moved out of the frame.
        if ball_body.position.y < 0:
            
            # Remove the ball from the space.
            space.remove(ball_body, ball_shape)
            
            # Remove the ball from the list.
            to_draw['Balls'].remove(ball)
            
    # Draw the created balls and the segments on the frame.
    frame = drawBalls(frame, to_draw['Balls'])
    frame = drawObstacles(frame, to_draw['Segments'])
    
    # Check if the FPS is > 0, to avoid division by zero.
    if frames_per_second > 0:
        
        # Step the simulation one step forward.
        space.step(1/frames_per_second)
    
    # Display the frame.
    cv2.imshow("Webcam Feed", frame)
    
    # Wait for 1ms. If a key is pressed, retreive the ASCII code of the key.
    k = cv2.waitKey(1) & 0xFF
    
    # Check if 'ESC' is pressed and break the loop.
    if k == 27:
        break
    
    # Check if the 'b' key is pressed.
    elif k == ord('b'):
        
        # Get radius (any random number between 30-70) of the ball to be created.
        ball_radius = np.random.choice(range(30, 70))  
        
        # Create a ball at a random x-coordinate value at the top of the frame.
        (ball_body, ball_shape), ball_color = createBall(frame, ball_position=(np.random.choice(range(0,width)),0), 
                                                         radius=ball_radius)
        
        # Append the ball's body, shape, radius, and color into the list.
        to_draw['Balls'].append((ball_body, ball_shape, ball_radius, ball_color))
        
        # Add both the body and the shape of the ball to the simulation.
        space.add(ball_body, ball_shape)

# Release the VideoCapture Object and close the windows.
camera_video.release()
cv2.destroyAllWindows()



# Additional comments:
#           In summary, we have two functions that will
#           create a ball and a function that will create
#           static objects so that it can act as obstacles.
#           
#           We record each ball that was created so that when it
#           goes below 0 in the y-axis, the ball can be deleted.
#           If this is not done, the ball will remain in the
#           memory even though it is no longer visible in the screen.

Working perfectly fine, as expected.

### **<font style="color:rgb(255,140,0)"> Code License Agreement </font>**
```
Copyright (c) 2022 Bleedai.com

Feel free to use this code for your own projects commercial or noncommercial, these projects can be Research-based, just for fun, for-profit, or even Education with the exception that you’re not going to use it for developing a course, book, guide, or any other educational products.

Under *NO CONDITION OR CIRCUMSTANCE* you may use this code for your own paid educational or self-promotional ventures without written consent from Taha Anwar (BleedAI.com).

```

